# AB Test with Snowpark Integration.

In [49]:
# grab our dependencies.

from snowflake.snowpark.session import Session
from snowflake.snowpark.types import IntegerType, StringType, StructType, FloatType, StructField, DateType, Variant
from snowflake.snowpark.functions import udf, sum, col, month, year, call_udf, lit
from snowflake.snowpark.version import VERSION

# instead of logging in through a json file.
import snowflake.connector as sf
from snowflake.connector.pandas_tools import write_pandas

import json
import pandas as pd
import logging

import sys
import numpy as np

logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)


In [25]:
# connect to snowflake through our connector. using single sign on.
conn = sf.connect(
            user = 'j.ongteco@xxxxxxxx',
            role = 'xx',
            account = 'xxxxxxx',
            warehouse = 'xxxxxx',
            database = "xxxxxx",
            schema = 'crm',
            authenticator = 'externalbrowser'
        )

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/6181f579-1e5e-4186-b49b-f22a2c2f7a99/saml2?SAMLRequest=lZJRb9owFIX%2FSuQ9J3EcoGABVQrtxtR1tMAm9c0kN%2BDVsTNfh7T79TOhSN1DK%2B3Nuj7H%2Fu49d3z5XKngABal0ROSRJQEoHNTSL2bkM36JhySAJ3QhVBGw4S8AJLL6RhFpWqeNW6vH%2BB3A%2BgC%2F5BG3l1MSGM1NwIlci0qQO5yvsq%2B3XIWUV5b40xuFHlj%2BdghEME6T3i2FCg93t65msdx27ZRm0bG7mJGKY3pKPaqo%2BTTWf%2Fse3pHn8S0d9R7hZcvX9mupD6N4COs7UmE%2FMt6vQyX31drEmRn1JnR2FRgV2APMofNw%2B0JAD3BfHkdiTpE4%2BcHAl3IItSmLZV4gtxUdeP8s5E%2FxSUUsTI76TtfzCekfpJFYfo%2F71m9%2Fzy%2F32R0lv55ekwHwurru6tfme1vN7j5ulWHPRSYk%2BDHOVp2jHaB2MBCHwN1vkRZGtJ%2ByHrrhHE65CmLhmnySIK5D1Rq4TrnmbrjiCqZW4OmdEYrqaGjHCTDpOxfjMIE%2BhD2kuEg3PZG27BkTLCclRdiNIqPMTNyWh3egdjpfw9kHL%2B1v67hnU9mMV8aJfOX4MbYSrj3g0uipKvIIiw7KYdKSJUVhQVEH6BSpp1ZEM5vu7MNkH

# python connector way

In [26]:
# set variables in a similar manner.
campaignid = '20227'
country = 'NL'

# actionids for stores.
flex_s = '1000500'
flex = '1000300'

start_date = '2023-04-22'

# write your main query and include the f before your query.
query = f"""

WITH s AS
(
         SELECT
                  CASE
                           WHEN _actionid = '{flex_s}' THEN 'FLEX_STORENAME'
                           WHEN _actionid = '{flex}' THEN 'FLEX'
                  END              AS s_actionid,
                  Count(_actionid) AS s_sent
         FROM     ods_eu.selligent.gen_flags
         WHERE    _campaignid = '{campaignid}'
         AND      listid = '{country}'
         AND      _dt BETWEEN '{start_date}' AND DATEADD(day, 1, '{start_date}')
         GROUP BY _actionid ), o AS
(
           SELECT
                      CASE
                                 WHEN _actionid = '{flex_s}' THEN 'FLEX_STORENAME'
                                 WHEN _actionid = '{flex}' THEN 'FLEX'
                      END                                      AS ord_actionid,
                      count(DISTINCT customer_email_keymapid)  AS customers,
                      count(order_status_upper)                AS orders,
                      round((orders / customers), 2)           AS frequency,
                      round(sum(royalty_sales_extax), 2)       AS revenue,
                      round(sum(food_theoretical_cost), 2)     AS food,
                      round(revenue - food, 2)                 AS sales_less_foodcosts,
                      round(revenue / customers, 2)            AS customer_revenue,
                      round(( revenue - food ) / customers, 2) AS customer_clv,
                      round(customer_revenue / frequency, 2)   AS avg_ticket
                      -- Round(stddev(royalty_sales_extax), 2)    AS std_dev_sales,
                      -- Abs(customer_revenue - avg_ticket) / std_dev_sales AS z_score_sales
           FROM       ods_eu.selligent.gen_users u
           INNER JOIN ods_eu.selligent.gen_flags b
           ON         u._userid = b._userid
           AND        u.listid = b.listid
           INNER JOIN ods_eu.selligent.gen_consent c
           ON         u._userid = c._userid
           AND        c.market = u.listid
           INNER JOIN mdl_store.orders_pos.orders o
           ON         u._email_address_keymapid = o.customer_email_keymapid
           AND        u.listid = o.countrycode
                      -- AND B._actionid = S._actionid
           WHERE      u.listid = '{country}'
           AND        b._campaignid = '{campaignid}'
           AND        c._status = 1
           AND        c._optin_id = 2
           AND        o.order_status_pos_code = 4
           AND        o.channel_group_upper = 'DIGITAL'
           AND        o.order_date BETWEEN '{start_date}' AND DATEADD(day, 1, '{start_date}')
           GROUP BY   ord_actionid--,  O.order_date is irrelevant if this was sent during the same day.

)
SELECT   ord_actionid AS id,
         s_sent AS sent,
         customers,
         orders,
         frequency as freq,
         revenue,
         customer_revenue,
         customer_clv,
         avg_ticket
FROM     s,
         o
WHERE    s_actionid = ord_actionid
GROUP BY id,
         sent,
         customers,
         orders,
         freq,
         revenue,
         customer_revenue,
         customer_clv,
         avg_ticket
ORDER BY id


"""
cur = conn.cursor()
cur.execute(query)
ab = cur.fetch_pandas_all()


In [27]:
ab

,ID,SENT,CUSTOMERS,ORDERS,FREQ,REVENUE,CUSTOMER_REVENUE,CUSTOMER_CLV,AVG_TICKET
0,FLEX,56283,1712,3000,1.75,70369.99,41.10,29.56,23.49
1,FLEX_STORENAME,56887,1704,3120,1.83,71876.43,42.18,30.17,23.05


In [28]:
print(type(ab))

<class 'pandas.core.frame.DataFrame'>


# Convert to Snowpark Dataframe

In [29]:
# this saves us a table inside snowflake called ab campaign 20227
success, nchunks, nrows, _ = write_pandas(conn, ab, "AB_CAMPAIGN_20227", auto_create_table=True)

What I learned is that its better to build a table outside of the notebook and then call the table through the notebook (through Session.table)

In [43]:
# create a session object from an existing python connector connection. 
# https://docs.snowflake.com/developer-guide/snowpark/reference/python/api/snowflake.snowpark.Session.html#snowflake.snowpark.Session
session = Session.builder.configs({"connection":conn}).create()

In [45]:
snowpark_df = session.table("AB_CAMPAIGN_20227")
snowpark_df # our snowflake dataframe.

In [53]:
# optimized!!!! 
print(f"Size in mb for Pandas DataFrame in memory: {np.round(sys.getsizeof(ab))}") 
print(f"Size in mb for Snowpark DataFrame in memory: {np.round(sys.getsizeof(snowpark_df))}") 

Size in mb for Pandas DataFrame in memory: 1444
Size in mb for Snowpark DataFrame in memory: 48


In [50]:
snowpark_df.show()

------------------------------------------------------------------------------------------------------------------------------
|"ID"            |"SENT"  |"CUSTOMERS"  |"ORDERS"  |"FREQ"  |"REVENUE"  |"CUSTOMER_REVENUE"  |"CUSTOMER_CLV"  |"AVG_TICKET"  |
------------------------------------------------------------------------------------------------------------------------------
|FLEX            |56283   |1712         |3000      |1.75    |70369.99   |41.10               |29.56           |23.49         |
|FLEX_STORENAME  |56887   |1704         |3120      |1.83    |71876.43   |42.18               |30.17           |23.05         |
------------------------------------------------------------------------------------------------------------------------------



In [56]:
# https://docs.snowflake.com/en/developer-guide/snowpark/python/working-with-dataframes
flex = session.table("AB_CAMPAIGN_20227").filter(col("id") == 'FLEX')
flex_storename = session.table("AB_CAMPAIGN_20227").filter(col("id") == 'FLEX_STORENAME')

In [57]:
flex_storename.show()

------------------------------------------------------------------------------------------------------------------------------
|"ID"            |"SENT"  |"CUSTOMERS"  |"ORDERS"  |"FREQ"  |"REVENUE"  |"CUSTOMER_REVENUE"  |"CUSTOMER_CLV"  |"AVG_TICKET"  |
------------------------------------------------------------------------------------------------------------------------------
|FLEX_STORENAME  |56887   |1704         |3120      |1.83    |71876.43   |42.18               |30.17           |23.05         |
------------------------------------------------------------------------------------------------------------------------------



In [ ]:
# to bring it back to pandas.

f = flex.select(col("ORDERS")).to_pandas()
f_s = flex_storename.select(col("ORDERS")).to_pandas()

So i think the idea here is to transform the data in python. Save it as a table in snowflake. Then run the model through a stored procedure.

# Voila!